This notebook is the second step in hyper-parameter search trying to find a model that can actually predict emissions. Based on the previous notebook, four models will be tested on each of the four pollutants:
* MAPE + LReLU + Adam + 20% dropout
* MAPE + PReLU + Adam + 20% dropout
* R2 + LReLU + Adam + 20% dropout
* R2 + PReLU + Adam + 20% dropout

**ALL** models will have a **ReLU** output. Although this is *not* standard practice, a massive source of error was that the past models were predicting **NEGATIVE** emissions. By having a ReLU output, values can only be negative when leaving the neural network. 

Each of these models will be trained trying to predict HC, CO, CO2, and NOx separately. This results in a total of 16 models to be trained and evaluated. As in the last notebook, the models will be evaluated using MSPE. 

In [1]:
from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, Dropout, advanced_activations, BatchNormalization, LeakyReLU, PReLU
from keras import losses, optimizers, activations
import keras.backend as K

import h5py

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.externals import joblib
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import time
import datetime
import os

C:\Users\drllc\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
output_path = os.path.join('.','output')

## Load Data

In [3]:
data_scaled_shuffled = pd.read_csv('Dataset_Scaled_Shuffled.csv')
print('Shuffled dataset loaded.')

Shuffled dataset loaded.


## Prepare Data

In [4]:
# Get number of data points
data_points = data_scaled_shuffled.shape[0]

# Set sizes for train, dev, test sets
train_percent = 0.8
train_size = round(train_percent*data_points)

if (data_points-train_size)%2 == 0:
    dev_size = int((data_points-train_size)/2)
    test_size = dev_size
    print('Train Size = {}'.format(train_size))
    print('Dev Size = {}'.format(dev_size))
    print('Test Size = {}'.format(test_size))
    print('Remainder = {}'.format(train_size+dev_size+test_size-data_points))
    
else:
    train_size = train_size-1
    dev_size = int((data_points-train_size)/2)
    test_size = dev_size 
    print('Train Size = {}'.format(train_size))
    print('Dev Size = {}'.format(dev_size))
    print('Test Size = {}'.format(test_size))
    print('Remainder = {}'.format(train_size+dev_size+test_size-data_points))

Train Size = 62511
Dev Size = 7814
Test Size = 7814
Remainder = 0


In [5]:
# Divide data into train, dev, and test sets
train_set = data_scaled_shuffled[:train_size]
dev_set = data_scaled_shuffled[train_size:train_size+dev_size]
test_set = data_scaled_shuffled[train_size+dev_size:train_size+dev_size+test_size]

# Reset index for all sets
train_set = train_set.reset_index(drop=True)
dev_set = dev_set.reset_index(drop=True)
test_set = test_set.reset_index(drop=True)

# Get values
train_set_values = train_set.values
dev_set_values = dev_set.values
test_set_values = test_set.values

# Number of emissions: HC, CO, CO2, NOX
n_out = 4

# SLICING: [start row:end row , start column:end column]
# Split into inputs and outputs
x_train = train_set_values[:,:-n_out]
x_dev = dev_set_values[:,:-n_out]
x_test = test_set_values[:,:-n_out]

emission_names = ['HC', 'CO', 'CO2', 'NOX']

def get_y_pollutant(name):

    HC_train = train_set_values[:,-n_out]
    CO_train = train_set_values[:,-n_out+1]
    CO2_train = train_set_values[:,-n_out+2]
    NOX_train = train_set_values[:,-n_out+3]

    HC_dev = dev_set_values[:,-n_out]
    CO_dev = dev_set_values[:,-n_out+1]
    CO2_dev = dev_set_values[:,-n_out+2]
    NOX_dev = dev_set_values[:,-n_out+3]

    HC_test = test_set_values[:,-n_out]
    CO_test = test_set_values[:,-n_out+1]
    CO2_test = test_set_values[:,-n_out+2]
    NOX_test = test_set_values[:,-n_out+3]

    if name == 'HC':
        y_train = HC_train
        y_dev = HC_dev
        y_test = HC_test
        
    if name == 'CO':
        y_train = CO_train
        y_dev = CO_dev
        y_test = CO_test
        
    if name == 'CO2':
        y_train = CO2_train
        y_dev = CO2_dev
        y_test = CO2_test
        
    if name == 'NOX':
        y_train = NOX_train
        y_dev = NOX_dev
        y_test = NOX_test
        
    return y_train, y_dev, y_test

## Inverse Scaling of Data

* This will be used later in the code to evaluate models

#### Import scalers

In [6]:
# Create an empty list to put all the scalers
scalers = []

for i in range(np.size(data_scaled_shuffled.columns)):
    
    scaler_filename = "Scalers/scaler{}.save".format(i)
    scaler = joblib.load(scaler_filename)
    
    scalers.append(scaler)

#### Inverse Scale Data

In [7]:
# First, inverse transform all original values from the test_set
test_set_inverse = test_set.copy()

for i in range(np.size(data_scaled_shuffled.columns)):
    
    col_name = data_scaled_shuffled.columns[i]
    
    values = test_set_inverse[col_name].values
    values = values.astype('float64')
    values = values.reshape(values.shape[0],1)
    
    test_set_inverse[col_name] = scalers[i].inverse_transform(values)
    
    print('Success with feature: {}'.format(col_name))

Success with feature: Year
Success with feature: Vehicle_Code
Success with feature: Manufacturer_Code
Success with feature: Displacement
Success with feature: Fuel_System
Success with feature: Gears
Success with feature: Transmission_Code
Success with feature: ETW
Success with feature: HP
Success with feature: Drive_System_Code
Success with feature: Fuel_Code
Success with feature: V_avg
Success with feature: V_max
Success with feature: V_std
Success with feature: a_pos
Success with feature: a_neg
Success with feature: Peak_pos
Success with feature: Peak_neg
Success with feature: HC
Success with feature: CO
Success with feature: CO2
Success with feature: Nox


-----------------
## Models

#### Basics

In [8]:
# Mini-batch size, epochs
batch_size = 64
epochs = 300

#### Define the R2 metric

In [9]:
# custom R2-score metrics for keras backend
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

#### Variables

In [10]:
# Loss functions to try
names_losses = ['MAPE','R2']    
dict_losses ={'MAPE': losses.mean_absolute_percentage_error,
             'R2': r2_keras}

#--------------------------------------------------------------------------------- 

# Activation functions to try
names_activations = ['LReLU', 'PReLU']
    # A function has to be called so that a new instance of the function can be created in each layer
def get_activation(name):

    if name == 'LReLU':
        function = advanced_activations.LeakyReLU()
    if name == 'PReLU':
        function = advanced_activations.PReLU()
        
    return function

#---------------------------------------------------------------------------------     

# Optimizers to be tried out
names_optimizers = ['Adam']
dict_optimizers ={'Adam': optimizers.Adam(amsgrad=False)}   

#--------------------------------------------------------------------------------- 

# Dropout rate to be tried
dropouts = [0.2]

#--------------------------------------------------------------------------------- 

print('Loss Functions = {}'.format(len(names_losses)))
print('Activation Functions = {}'.format(len(names_activations)))
print('Optimizers = {}'.format(len(names_optimizers)))
print('Dropout Rates = {}'.format(len(dropouts)))
print('--------------------------------')
print('Number of models for test = {}'.format(len(names_losses)*len(names_activations)*len(names_optimizers)*len(dropouts)))

Loss Functions = 2
Activation Functions = 2
Optimizers = 1
Dropout Rates = 1
--------------------------------
Number of models for test = 4


#### Build Model

In [11]:
def build_model(number, loss_func, activation_name, optimizer, dd):
    
    # Create model
    model = Sequential(name='Model_{}'.format(number))

    model.add(Dense(256, input_dim=x_train.shape[1]))
    model.add(get_activation(activation_name))
    model.add(Dropout(dd))
    model.add(BatchNormalization())

    model.add(Dense(128))
    model.add(get_activation(activation_name))
    model.add(Dropout(dd))
    model.add(BatchNormalization())

    model.add(Dense(64))
    model.add(get_activation(activation_name))
    model.add(Dropout(dd))
    model.add(BatchNormalization())

    model.add(Dense(32))
    model.add(get_activation(activation_name))
    model.add(Dropout(dd))
    model.add(BatchNormalization())

    model.add(Dense(16))
    model.add(get_activation(activation_name))
    model.add(Dropout(dd))
    model.add(BatchNormalization())

    model.add(Dense(1))
    model.add(advanced_activations.ReLU())

    #Compile model
    model.compile(loss=loss_func, optimizer=optimizer, metrics = ['accuracy'])
    
    print('{} Created'.format(model.name))
    print('----------------------------------')
    
    return model

#### Train Model

In [12]:
def train_models(model, y_train, y_dev):
    
    print('{} - Training'.format(model.name))
    print('- Started on {} at {}'.format(str(datetime.datetime.now())[5:-16], str(datetime.datetime.now())[11:-10]))
    # Start timer
    start_time = time.time()

    # fit network
    history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, 
                        validation_data=(x_dev, y_dev), verbose=0, shuffle=True)

    # End timer
    end_time = time.time() - start_time
    print('{} - Training Complete'.format(model.name))
    print('- Time: {:.3f} min'.format(end_time/60))
    print('- Loss = {:.5f}'.format(history.history['loss'][-1]))
    print('- Val Loss = {:.5f}'.format(history.history['val_loss'][-1]))
    print('----------------------------------')
        
    return history

#### Make Predictions and Calculate Error

In [13]:
# Function to define MSPE
def msp_error(true,pred):
    error = 100*np.sum(((true-pred)/true)**2)/np.size(true)
    return error

In [18]:
def predict_get_error(model, pollutant):
    
    print('Predicting with {}'.format(model.name))
    scaled_predictions = model.predict(x_test)
    
    print('Inverse Scaling Operation') 
    
    if pollutant == 'HC':   
        # Inverse the scaling operation on the predictions
        predictions = scalers[-4].inverse_transform(scaled_predictions)
        
        print('Calculating {} Error'.format(pollutant))
        mspe = msp_error(test_set_inverse['HC'].values, predictions)
        
    if pollutant == 'CO':   
        # Inverse the scaling operation on the predictions
        predictions = scalers[-3].inverse_transform(scaled_predictions)
        
        print('Calculating {} Error'.format(pollutant))
        mspe = msp_error(test_set_inverse['CO'].values, predictions)
        
    if pollutant == 'CO2':   
        # Inverse the scaling operation on the predictions
        predictions = scalers[-2].inverse_transform(scaled_predictions)
        
        print('Calculating {} Error'.format(pollutant))
        mspe = msp_error(test_set_inverse['CO2'].values, predictions)
        
    if pollutant == 'NOX':   
        # Inverse the scaling operation on the predictions
        predictions = scalers[-1].inverse_transform(scaled_predictions)
        
        print('Calculating {} Error'.format(pollutant))
        mspe = msp_error(test_set_inverse['Nox'].values, predictions)
        
    print('- {} Error  = {:.2e}'.format(pollutant,mspe))
    print('----------------------------------')
    
    return mspe

#### Process Models and Rank with MSPE

In [16]:
def process_models():
    
    count = 1
    model_list = []
    history_list = []
    HC_error_list = []
    CO_error_list = []
    CO2_error_list = []
    NOX_error_list = []

    for loss_name in names_losses:
        loss = dict_losses[loss_name]

        for activation_name in names_activations:

            for optimizer_name in names_optimizers:
                optimizer = dict_optimizers[optimizer_name]

                for dd in dropouts:
                    
                    for pollutant in emission_names:

                        # Print model variables
                        print('Model_{} Variables:'.format(count))
                        print('- Loss: {}'.format(loss_name))
                        print('- Activation: {}'.format(activation_name))
                        print('- Optimizer: {}'.format(optimizer_name))
                        print('- Dropout: {}%'.format(dd*100))
                        print('- Pollutant: {}'.format(pollutant))
                        print('----------------------------------')
                        
                        # Get output values
                        y_train, y_dev, y_test = get_y_pollutant(pollutant)

                        # Create model
                        model = build_model(count,loss,activation_name,optimizer,dd)

                        # Train model
                        history = train_models(model, y_train, y_dev)
                        history_list.append(history)

                        # Make predictions and calculate error
                        error = predict_get_error(model, pollutant)
                        
                        if pollutant == 'HC':   
                            HC_error_list.append([model.name, loss_name, 
                                                  activation_name, optimizer_name, dd, pollutant, error])

                        if pollutant == 'CO':   
                            CO_error_list.append([model.name, loss_name, 
                                                  activation_name, optimizer_name, dd, pollutant, error])

                        if pollutant == 'CO2':   
                            CO2_error_list.append([model.name, loss_name, 
                                                  activation_name, optimizer_name, dd, pollutant, error])

                        if pollutant == 'NOX':
                            NOX_error_list.append([model.name, loss_name, 
                                                  activation_name, optimizer_name, dd, pollutant, error])
                        
                        # Announce one model process ended
                        print('============== MODEL {} PROCESS END =============='.format(count))
                        print(' ')

                        # Increase counter by 1
                        count = count+1
                        
                        # Add TRAINED model to list
                        model_list.append(model)

    print('Creating DataFrame')                
    HC_error = pd.DataFrame(HC_error_list)
    CO_error = pd.DataFrame(CO_error_list)
    CO2_error = pd.DataFrame(CO2_error_list)
    NOX_error = pd.DataFrame(NOX_error_list)

    print('Changing DataFrame column names')
    HC_error.columns = ['Model', 'Loss', 'Activation', 'Optimizer', 'Dropout', 'Pollutant', 'MSPE']
    CO_error.columns = ['Model', 'Loss', 'Activation', 'Optimizer', 'Dropout', 'Pollutant', 'MSPE']
    CO2_error.columns = ['Model', 'Loss', 'Activation', 'Optimizer', 'Dropout', 'Pollutant', 'MSPE']
    NOX_error.columns = ['Model', 'Loss', 'Activation', 'Optimizer', 'Dropout', 'Pollutant', 'MSPE']

    print('Ranking Models')
    HC_error.sort_values(by=['MSPE'], inplace=True)
    CO_error.sort_values(by=['MSPE'], inplace=True)
    CO2_error.sort_values(by=['MSPE'], inplace=True)
    NOX_error.sort_values(by=['MSPE'], inplace=True)

    count = 0
    
    return HC_error, CO_error, CO2_error, NOX_error, model_list, history_list

In [19]:
HC_ranking, CO_ranking, CO2_ranking, NOX_ranking, models, histories = process_models()

Model_1 Variables:
- Loss: MAPE
- Activation: LReLU
- Optimizer: Adam
- Dropout: 20.0%
- Pollutant: HC
----------------------------------
Model_1 Created
----------------------------------
Model_1 - Training
- Started on 03-25 at 23:19
Model_1 - Training Complete
- Time: 39.640 min
- Loss = 100.00000
- Val Loss = 100.00000
----------------------------------
Predicting with Model_1
Inverse Scaling Operation
Calculating HC Error
- HC Error  = 7.80e+05
----------------------------------
============== MODEL 1 PROCESS END ==============
 
Model_2 Variables:
- Loss: MAPE
- Activation: LReLU
- Optimizer: Adam
- Dropout: 20.0%
- Pollutant: CO
----------------------------------
Model_2 Created
----------------------------------
Model_2 - Training
- Started on 03-25 at 23:59
Model_2 - Training Complete
- Time: 40.041 min
- Loss = 99.97600
- Val Loss = 99.97440
----------------------------------
Predicting with Model_2
Inverse Scaling Operation
Calculating CO Error
- CO Error  = 7.76e+05
-------

- CO2 Error  = 4.95e+19
----------------------------------
============== MODEL 15 PROCESS END ==============
 
Model_16 Variables:
- Loss: R2
- Activation: PReLU
- Optimizer: Adam
- Dropout: 20.0%
- Pollutant: NOX
----------------------------------
Model_16 Created
----------------------------------
Model_16 - Training
- Started on 03-26 at 09:50
Model_16 - Training Complete
- Time: 44.539 min
- Loss = -741360362877866.75000
- Val Loss = -756518267723786.50000
----------------------------------
Predicting with Model_16
Inverse Scaling Operation
Calculating NOX Error
- NOX Error  = 3.33e+26
----------------------------------
============== MODEL 16 PROCESS END ==============
 
Creating DataFrame
Changing DataFrame column names
Ranking Models


In [20]:
HC_ranking

,Model,Loss,Activation,Optimizer,Dropout,Pollutant,MSPE
0,Model_1,MAPE,LReLU,Adam,0.2,HC,7.801466e+05
1,Model_5,MAPE,PReLU,Adam,0.2,HC,7.801466e+05
2,Model_9,R2,LReLU,Adam,0.2,HC,5.867494e+25
3,Model_13,R2,PReLU,Adam,0.2,HC,6.087465e+25


In [21]:
CO_ranking

,Model,Loss,Activation,Optimizer,Dropout,Pollutant,MSPE
0,Model_2,MAPE,LReLU,Adam,0.2,CO,7.764241e+05
1,Model_6,MAPE,PReLU,Adam,0.2,CO,7.764241e+05
2,Model_10,R2,LReLU,Adam,0.2,CO,5.287206e+24
3,Model_14,R2,PReLU,Adam,0.2,CO,5.438584e+24


In [22]:
CO2_ranking

,Model,Loss,Activation,Optimizer,Dropout,Pollutant,MSPE
0,Model_3,MAPE,LReLU,Adam,0.2,CO2,1.367073e+05
1,Model_7,MAPE,PReLU,Adam,0.2,CO2,3.832736e+05
2,Model_11,R2,LReLU,Adam,0.2,CO2,4.521210e+19
3,Model_15,R2,PReLU,Adam,0.2,CO2,4.949186e+19


In [23]:
NOX_ranking

,Model,Loss,Activation,Optimizer,Dropout,Pollutant,MSPE
0,Model_4,MAPE,LReLU,Adam,0.2,NOX,3.922313e+11
1,Model_8,MAPE,PReLU,Adam,0.2,NOX,5.369868e+12
2,Model_12,R2,LReLU,Adam,0.2,NOX,3.222579e+26
3,Model_16,R2,PReLU,Adam,0.2,NOX,3.327281e+26


In [25]:
overall_ranking = pd.concat([HC_ranking,CO2_ranking,CO_ranking,NOX_ranking])

In [27]:
overall_ranking.sort_values(by='MSPE', inplace=True)
overall_ranking

,Model,Loss,Activation,Optimizer,Dropout,Pollutant,MSPE
0,Model_3,MAPE,LReLU,Adam,0.2,CO2,1.367073e+05
1,Model_7,MAPE,PReLU,Adam,0.2,CO2,3.832736e+05
0,Model_2,MAPE,LReLU,Adam,0.2,CO,7.764241e+05
1,Model_6,MAPE,PReLU,Adam,0.2,CO,7.764241e+05
0,Model_1,MAPE,LReLU,Adam,0.2,HC,7.801466e+05
1,Model_5,MAPE,PReLU,Adam,0.2,HC,7.801466e+05
0,Model_4,MAPE,LReLU,Adam,0.2,NOX,3.922313e+11
1,Model_8,MAPE,PReLU,Adam,0.2,NOX,5.369868e+12
2,Model_11,R2,LReLU,Adam,0.2,CO2,4.521210e+19
3,Model_15,R2,PReLU,Adam,0.2,CO2,4.949186e+19


In [29]:
epoch_vector = np.linspace(1,epochs,epochs)

for i in [2,1,0,3]:
    
    model = models[i]
    history = histories[i]
    
    model.save(os.path.join(output_path,'{}.h5'.format(model.name)))
    
    
    hist_data = [epoch_vector, history.history['loss'], history.history['val_loss']]
    
    hist_data = pd.DataFrame(hist_data).transpose()
    
    # Change column names
    hist_data.columns = ['Epochs','loss','val_loss']
    
    hist_data.to_csv(os.path.join(output_path,'Training_History_{}.csv'.format(model.name)), 
                     encoding='utf-8', index=False)

In [28]:
overall_ranking.to_csv(os.path.join(output_path,'Model_Ranking.csv'), encoding='utf-8', index=False)

In [42]:
number = 10
prediction_test = models[number].predict(x_test)

In [44]:
print('{}'.format(models[number].name))
print('Max = {}'.format(np.max(prediction_test)))
print('Min = {}'.format(np.min(prediction_test)))
print('Mean = {}'.format(np.mean(prediction_test)))

Model_11
Max = 3502612.5
Min = 410055.4375
Mean = 1413129.75


## Next Steps

All the data will be moved from the *output* folder to the *Gen_4* folder. Top model for each pollutant was saved. 

All models trained with R2 are completely discarded, they are useless and won't even be mentioned in the end.

Without a doubt, the best performing models (according to Mean Square Percent Error) were:
* MAPE + LeakyReLU + Adam + ReLU Output

The outputs being predicted by the neural networks with **MAPE** loss function are **ZERO**, which makes no sense. Because Gen_3 models were predicting negative numbers, the error for these is smaller, but the problem persists. 

The models that use R2 function are basically useless, but the outputs are at least positive. 

This **zeros** problem can come from the fact that negative values are appearing inside the neural network and when they get to the output layer - which has a ReLU function - they instantly become zero. 

There's several approaches:
* Try one final type of linear activation unit called ELU. According to Krishna in https://medium.com/@krishnakalyan3/introduction-to-exponential-linear-unit-d3e2904b366c, this activation function outperforms others in tests. 
    * Using ELU could make the negative predictions appear again. 
* Try a sigmoid activation function in the last layer. It's not standard practice, but could mix up results. It is likely that values will be driven to zero here aswell. 
* Try using ReLU in **every** layer. This could prevent the negative values inside the network, but could increase the appearance of zeros and having a lot of dead neurons. 
* One of the input parameters could be introducing a negative correlation with the output data and damaging the learning process. **Probably the most sensible next step would be to remove the input variables one at a time and train models using 17 of the 18 inputs.** If nothing changes, then test removing two at a time (but this would increase the number of models too much).
    * Removing one input variable at a time produces 18 different data sets, which means 18 different models. 
    * Removing two input variables at a time produces **306** different data sets. Too much to test.